In [0]:
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

spark = SparkSession.builder.getOrCreate()

In [0]:
%pip install --upgrade openai

Python interpreter will be restarted.
  Using cached openai-1.14.2-py3-none-any.whl (262 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
  Using cached httpcore-1.0.4-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: distro
    Found existing installation: distro 1.4.0
    Not uninstalling distro at /usr/lib/python3/dist-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-c41ce6e9-dc06-4a70-9ebe-0de1662d6e3b
    Can't uninstall 'distro'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
%pip install google.generativeai

Python interpreter will be restarted.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.5
    Not uninstalling protobuf at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-d685a978-305c-4186-9919-171b28984712
    Can't uninstall 'protobuf'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
import google.generativeai as genai
GOOGLE_API_KEY = 'YOUR_GEMINI_API_KEY'
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# Load Data

In [0]:
users_skills = spark.read.parquet("dbfs:/FileStore/shared_uploads/kinani@campus.technion.ac.il/users_skills_new.parquet")

In [0]:
from pyspark.sql.functions import size, col, length
profiles = spark.read.parquet('/linkedin/people')
# print(profiles.count())

In [0]:
companies = spark.read.parquet('/linkedin/companies')
print(companies.count())

1422797


# Using Gemini  to get user skills

In [0]:
def skills_list(str_list,position):
    dict_skills ={}
    s_list = [skill.strip()[2:] for skill in str_list.strip().split('\n') if skill]
    s_list = [skill.split('(')[0].strip() for skill in s_list if skill ]
    if len(s_list) > 7 :
        skill = []
        for i in range(len(s_list)):
            if i%8 != 0 :
                skill.append(s_list[i])
            if (i >= 8 and i%8 == 0) :
                dict_skills[s_list[i-8][:-2]] = skill
                skill = []
                dict_skills[s_list[i][:-2]] = skill
    else:
        dict_skills[position] = s_list
    return dict_skills

In [0]:
from pyspark.sql.functions import size, col, length

df = profiles.filter(col('about').isNotNull())
df = df.where((length(col('about'))>10))
print(df.count())
train_dataset = df.where((length(col('position'))>2) & (~col('position').contains('Retired')))
print(train_dataset.count())
sampled_df = train_dataset.sample(False, 5500.0/train_dataset.count(), seed=None)
print(sampled_df.count())

726173
688578
4060


In [0]:
df=sampled_df.select('id','position')

In [0]:
ids = [id[0] for id in df.select("id").collect()]
positions = [position[0] for position in df.select("position").collect()]
list_position = []
for i in range(len(positions)):
    # print(i)
    try:
        position = positions[i]
        q = "tell me the position name based on this text if you can't determine a specific job title  write me only the word  null:" + position
        response = model.generate_content(q)
        pos = response.text
        if pos != 'null':
            q = 'based on this position name ' + response.text + 'tell me top 7 skills only names this person have'
            response = model.generate_content(q)
            dict_skill = skills_list(response.text,pos)
            list_position.append(dict_skill)
        else:
            list_position.append({'null': ['null']})
    except:
       list_position.append({i: []}) 

In [0]:
from pyspark.sql.types import MapType, LongType, StructType, StructField, StringType, ArrayType
print(len(list_position))
data = list(zip(ids, positions, list_position))

schema = StructType([
    StructField('id', StringType(), nullable=False),
    StructField('position', StringType(), nullable=False),
    StructField('skills', MapType(StringType(), ArrayType(StringType())), nullable=True)
])
df_users_skills = spark.createDataFrame(data, schema=schema)

4004


In [0]:
df_users_skills.display()

id,position,skills
connor-dixon-27b7b1264,Student at the Dan Patrick School of Sportscasting,Map(null -> List(null))
maryankaalsina,Program Manager at District Dance Academy. DDA is a dance studio and event venue servicing the Tampa Bay area.,"Map(Program Manager -> List(Project Management, Requirements Gathering, Stakeholder Management, Resource Management, Budget Management, Risk Management, Communication))"
selamwg,Motivation is the key to start something but being persistence gets you to the finish line.,Map(null -> List(null))
matthew-von-brock-1323554,Visual Effects Supervisor / Creative Director / VFX & Games Leadership open to work / Member of Visual Effects Society,"Map(Visual Effects Supervisor -> List(Advanced knowledge of visual effects software and techniques, Strong understanding of computer graphics and digital imaging, Ability to supervise and coordinate a team of visual effects artists, Excellent communication and presentation skills, Proven experience in managing complex visual effects projects, Deep understanding of filmmaking and cinematography, Passion for creating innovative and visually stunning effects))"
shailesh-dalwadi-pmp-b8826012,Project Manager at UPS,"Map(Project Manager -> List(Project Management, Stakeholder Management, Budget Management, Risk Management, Change Management, Communication, Leadership))"
valerie-valerian,"Strategic Consulting Associate at KHA, PLLC | UNT BS/MS in Accounting and Student Athlete Alumni","Map(Strategic Consulting Associate -> List(Strategic Planning, Business Analysis, Problem Solving, Communication, Presentation, Analytical Thinking, Project Management))"
michael-uren-8293b319a,Personal Trainer,"Map(Personal Trainer -> List(Exercise Physiology, Biomechanics, Exercise Training, Nutrition Counseling, Communication, Motivation, CPR and First Aid))"
lily-ganz-b539b9163,Emergency Medicine Physician Assistant,"Map(Physician Assistant -> List(Medical Diagnosis, Medical Examination, Patient Care, Prescription Writing, Surgery Assistance, Trauma Care, Emergency Medicine))"
vicente-valdez-44b659a5,Senior Customer Success Manager at Media Temple,"Map(Senior Customer Success Manager -> List(Strategic Planning, Customer Relationship Management, Communication, Data Analysis, Problem Solving, Stakeholder Management, Leadership))"
jan-keuchel-01403442,QUALITY SPECIALIST RN at Madonna Rehabilitation Hospitals,"Map(QUALITY SPECIALIST RN -> List(Quality Assurance, Risk Management, Clinical Documentation, Patient Safety, Regulatory Compliance, Process Improvement, Data Analysis))"


# Filtering the data

In [0]:
from pyspark.sql.functions import col, size, udf
import ast
from pyspark.sql.types import MapType, LongType, StructType, StructField, StringType, ArrayType, IntegerType

def extract_dict(dict_str):
    try:
        return ast.literal_eval(dict_str)
    except (SyntaxError, ValueError):
        dict_ = {}
        return dict_
      
extract_dict_udf = udf(extract_dict, MapType(StringType(), ArrayType(StringType())))
df_with_skills = users_skills.withColumn("skills", extract_dict_udf(col('skills')))
res = df_with_skills.where(size(col('skills')) == 0)
print(res.count())

def exception_solve(dict_int):
    if len(list(dict_int.keys())) > 0:
        if list(dict_int.keys())[0].isdigit():
            return True
        else:
            return False
    return False

res1 = df_with_skills.where(udf(exception_solve)(col('skills')) == True)
print(res1.count())

def null_solve(dict_int):
    if len(list(dict_int.keys())) > 0:
        if list(dict_int.keys())[0] == 'null':
            return True
        else:
            return False
    return False

res2 = df_with_skills.where(udf(null_solve)(col('skills')) == True)
print(res2.count())

0
14
1047


In [0]:
bad_links = [link[0] for link in res.select("id").collect()] + [link[0] for link in res1.select("id").collect()] + [link[0] for link in res2.select("id").collect()]
temp1 = users_skills.where(~col('id').isin(bad_links))
temp1.count()

Out[16]: 2940